In [ ]:
!pip install transformers datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import datasets
twitch_dataset = datasets.load_dataset('csv', data_files='/content/drive/MyDrive/twitch_dataset_with_head.csv')

twitch_dataset = twitch_dataset.filter(lambda x: isinstance(x['text'], str))

import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset

model_checkpoint = "distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def tokenize_function(examples):
    return tokenizer(examples["text"], max_length=tokenizer.model_max_length, truncation=True)

tokenized_datasets = twitch_dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text", "video_id"])

print(tokenized_datasets)

lm_datasets = tokenized_datasets

block_size = tokenizer.model_max_length

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

lm_datasets = tokenized_datasets

Mounted at /content/drive


Using custom data configuration default-317340911501bb94


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-317340911501bb94/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


...

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids'],
        num_rows: 66495627
    })
})


In [ ]:
import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset

from transformers import AutoConfig, AutoModelForMaskedLM
model_checkpoint = "distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_config(config)

In [ ]:
training_args = TrainingArguments(
    "drive/MyDrive/twitch_roberta/test-clm",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size = 8,
    push_to_hub=False,
    evaluation_strategy = 'steps',
    eval_steps = 10000,
    save_strategy = 'steps',
    save_steps = 20000,
    save_total_limit = 5
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
lm_datasets = lm_datasets["train"].train_test_split(0.001)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets['train'],
    eval_dataset=lm_datasets['test'],
    data_collator=data_collator,
)

In [ ]:
trainer.train(resume_from_checkpoint=True)

Loading model from drive/MyDrive/twitch_roberta/test-clm/checkpoint-400000).
***** Running training *****
  Num examples = 66429131
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 24910926
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 0
  Continuing training from global step 400000
  Will skip the first 0 epochs then the first 400000 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/400000 [00:00<?, ?it/s]

Step,Training Loss,Validation Loss
410000,2.696200,2.552453
420000,2.728200,2.545409
430000,2.614700,2.520029
440000,2.638700,2.540171
450000,2.649500,2.533763
460000,2.612800,2.536160
470000,2.627700,2.525440
480000,2.597300,2.543976
490000,2.634700,2.486709
500000,2.683700,2.496139


***** Running Evaluation *****
  Num examples = 66496
  Batch size = 8
***** Running Evaluation *****
  Num examples = 66496
  Batch size = 8
Saving model checkpoint to drive/MyDrive/twitch_roberta/test-clm/checkpoint-420000
Configuration saved in drive/MyDrive/twitch_roberta/test-clm/checkpoint-420000/config.json
Model weights saved in drive/MyDrive/twitch_roberta/test-clm/checkpoint-420000/pytorch_model.bin
Deleting older checkpoint [drive/MyDrive/twitch_roberta/test-clm/checkpoint-400000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 66496
  Batch size = 8
***** Running Evaluation *****
  Num examples = 66496
  Batch size = 8
Saving model checkpoint to drive/MyDrive/twitch_roberta/test-clm/checkpoint-440000
Configuration saved in drive/MyDrive/twitch_roberta/test-clm/checkpoint-440000/config.json
Model weights saved in drive/MyDrive/twitch_roberta/test-clm/checkpoint-440000/pytorch_model.bin
Deleting older checkpoint [drive/MyDrive/twitch_roberta/test-